In [2]:
%run ./PokemonLib.ipynb

In [31]:
def gatherDataForArena(locations, game, generation, starters, leaders, FISHING,specialPokemon):
    areas = produceAreasForLocations(locations)
    addEncounterablePokemonsToAreas(areas, FISHING)
    
    catchablePokemons = producePokemonForAreas(areas, generation)
    appendPokemonFromStarters(starters, catchablePokemons, generation)
    appendPokemonFromSpecial(specialPokemon, catchablePokemons, game, generation)
    appendPokemonFromEvolutions(catchablePokemons, game, generation)
    
    
    arenaLeaders = produceArenaLeaders(leaders, generation)

    return {'areas': areas, 'catchablePokemons': catchablePokemons, 'arenaLeaders': arenaLeaders}

In [9]:
def findEvolutionsRecursive(evolvesTo, originalName, game):
    entwicklungen = []
    for entwicklung in evolvesTo:
        entwicklungsName = entwicklung['species']['name']
        entwicklungPokeJson = requestPokemon(entwicklungsName)
        pokeIsInWantedGame = False
        for gameIndice in entwicklungPokeJson['game_indices']:
            if gameIndice['version']['name'] == game:
                pokeIsInWantedGame = True
        if((entwicklungsName != originalName) & pokeIsInWantedGame):
          #  print(originalName)
          #  print('entwickelt sich zu')
          #  print(entwicklungsName)
            evoDetails = entwicklung['evolution_details'][0]
            trigger = evoDetails['trigger']['name']
            if trigger == 'level-up':
                levelUpCondition = evoDetails['min_level']
            elif trigger == 'use-item':
                levelUpCondition = evoDetails['item']['name']
            elif trigger == 'trade':
                levelUpCondition = 'trade'
            else:
                levelUpCondition = None
           # print(levelUpCondition)
           # print('----')
            entwicklungen.append((entwicklungsName, levelUpCondition))
            #print(entwicklungen)
        if(entwicklung['evolves_to']!= None):
            entwicklungInDepth = findEvolutionsRecursive(entwicklung['evolves_to'], originalName, game)
            if(len(entwicklungInDepth)>0):
                for entw in entwicklungInDepth:
                    entwicklungen.append(entw)
    return entwicklungen


In [10]:
def fetchPokemonEvolutionPossibilities(catchablePokemons, game):
    allEvolves = []
    for poke in catchablePokemons:
        originalName = catchablePokemons[poke].name
        originalEncounterCondition = catchablePokemons[poke].encounterOrEvolutionTrigger
        pokeJson = fetchPokemon(originalName)
        speciesUrl = pokeJson['species']['url']
        speciesJson = requestFullUrl(speciesUrl)
        evolutionChainUrl = speciesJson['evolution_chain']['url']
        evolutionChainJson = requestFullUrl(evolutionChainUrl)
        chain = evolutionChainJson['chain']
        tempEvolvesTo = chain['evolves_to']
        evolves = findEvolutionsRecursive(tempEvolvesTo, originalName, game)
        for evolve in evolves:
            evolve = (evolve[0], evolve[1], originalEncounterCondition)
            allEvolves.append(evolve)
    return allEvolves

In [11]:
def appendPokemonFromEvolutions(catchablePokemons, game, generation):
    tempEvos = fetchPokemonEvolutionPossibilities(catchablePokemons, game)
    
    for evolution in tempEvos:
        evoName = evolution[0]
        if not evoName in catchablePokemons:
            catchablePokemons[evoName] = producePokemonFromEvolution(evolution, generation)

In [12]:
def appendPokemonFromSpecial(specialPokemon,catchablePokemons, game, generation):
    for name in specialPokemon:
        if not name in catchablePokemons:
            catchablePokemons[name] = producePokemon(name,generation, 'special')

In [13]:
def appendPokemonFromStarters(starters, catchablePokemons, generation):
     for name in starters:
        if not name in catchablePokemons:
            catchablePokemons[name] = producePokemon(name,generation, 'starter')

In [14]:
def produceAreasForLocation(location, steps):
    areasJson = fetchAreasToLocation(location)
    areas = []
    for areaJson in areasJson:
        area = Area(areaJson['name'], 0.2, areaJson, steps )
        areas.append(area)
    return areas

In [15]:
def produceAreasForLocations(locations):
    areas = []
    for location in locations:
        locationName = location['name']
        steps = location['steps']
        areasFromLocation = produceAreasForLocation(locationName, steps)
        for area in areasFromLocation:
            areas.append(area)
    return areas

In [16]:
def addEncounterablePokemonsToArea(area, FISHING):
     area.encounterablePokemons = returnPokemonAndChanceAndEncounterTypeFromArea(area.json ,[GAME], FISHING)

In [17]:
def addEncounterablePokemonsToAreas(areas, FISHING):
    for area in areas:
        addEncounterablePokemonsToArea(area, FISHING)
        area.calculateAverageEncounters()

In [18]:
def producePokemonForArea(area, pokemonSet, generation):
    for pokemon in area.encounterablePokemons:
        name = pokemon[0]
        encounterType = pokemon[2]
        if not name in pokemonSet:
            pokemonSet[name] = producePokemon(name, generation, encounterType )

In [19]:
def producePokemonForAreas(areas, generation):
    pokemonSet = {}
    for area in areas:
        producePokemonForArea(area, pokemonSet, generation)
    return pokemonSet

In [20]:
def producePokemon(name, generation, encounterType):
    moves = fetchPokemonMoves(name, generation)
    pokemonInstance = Pokemon(name, moves, encounterType)
    return pokemonInstance

In [21]:
def producePokemonFromEvolution(evolution, generation):
    evoName = evolution[0]
    evoTrigger = evolution[1]
    evoOrigin = evolution[2]
    moves = fetchPokemonMoves(evoName, generation)
    pokemonInstance = Pokemon(evoName, moves, evoTrigger)
    pokemonInstance.setOriginIfEvolution(evoOrigin)
    return pokemonInstance

In [22]:
def producePokemonWithCustomMoves(name, generation, customMoves):
    moves = []
    for move in customMoves:
        moves.append(produceMove(move))
    pokemonInstance = Pokemon(name, moves, 'encounter')
    return pokemonInstance

In [23]:
#pokemonTeam = [{name: 'pikachu', level: 12, moves:['tackle']}]
#produceArenaLeader('brock', 'red-blue', [{'name': 'pikachu', 'level': 12, 'moves':['tackle']}])
def produceArenaLeader(name, generation, pokemonTeam):
    pokemon = []
    for poke in pokemonTeam:
        producedPoke = producePokemonWithCustomMoves(poke['name'],generation,poke['moves'])
        pokemon.append((producedPoke, poke['level']))
    
    leader = ArenaLeader(name, pokemon)
    return leader

In [24]:
def produceArenaLeaders(leaders,generation):
    csvLeaders = []
    for leader in leaders:
        leader = produceArenaLeader(leader[0], generation, leader[1])
        csvLeaders.append(leader)
    return csvLeaders